In [196]:
import os
import re
import random
import glob

import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.probability import ConditionalFreqDist 
from nltk.probability import FreqDist 
nltk.download('stopwords')
nltk.download('punkt')
set(stopwords.words('spanish'))
nltk.download('wordnet')


# visualization imports
from IPython.display import display
import io
 

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aideenf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/aideenf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/aideenf/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [197]:
FLAG__AGGREGATE_CONVENTIONS_SENTENCES_FROM_GITHUB = True
FLAG__GENERATE_SENTECE_TOKENIZED_DOCS=True
FLAG__CREATE_KEYWORDS = True
FLAG__ON_BINDER = False


box_gstd = widgets.Checkbox(True, description='FLAG__GENERATE_SENTECE_TOKENIZED_DOCS')
box_acsfg = widgets.Checkbox(True, description='FLAG__AGGREGATE_CONVENTIONS_SENTENCES_FROM_GITHUB')
box_binder = widgets.Checkbox(False, description='Running on binder')

def update_box_gstd(change):
    global FLAG__GENERATE_SENTECE_TOKENIZED_DOCS
    FLAG__GENERATE_SENTECE_TOKENIZED_DOCS= change['new']
box_gstd.observe(update_box_gstd, 'value')

def update_box_acsfg(change):
    global FLAG__AGGREGATE_CONVENTIONS_SENTENCES_FROM_GITHUB
    FLAG__AGGREGATE_CONVENTIONS_SENTENCES_FROM_GITHUB = change['new']

box_acsfg.observe(update_box_acsfg, 'value')

def update_box_binder(change):
    global FLAG__ON_BINDER
    FLAG__ON_BINDER = change['new']

box_binder.observe(update_box_binder, 'value')

#display(box_acsfg, box_gstd, filePath1, filePath2, filePath3)
display(box_acsfg, box_gstd, box_binder)

Checkbox(value=True, description='FLAG__AGGREGATE_CONVENTIONS_SENTENCES_FROM_GITHUB')

Checkbox(value=True, description='FLAG__GENERATE_SENTECE_TOKENIZED_DOCS')

Checkbox(value=False, description='Running on binder')

In [198]:
if FLAG__ON_BINDER == True:
    ROOT_DIR = "/home/jovyan/cp_wssc"
else:
    ROOT_DIR = "."
    
print (ROOT_DIR)

.


In [199]:
BASE_DIR_SCI_ARTICLES = ROOT_DIR + "/Data/VALIDATED_DATA/Conventions/scientific_articles/"
BASE_DIR_GIT_HUB =  ROOT_DIR + "/Data/VALIDATED_DATA/Conventions/github"
BASE_DIR_AGGREGATED =  ROOT_DIR + "/Data/VALIDATED_DATA/Conventions/aggregated/"
KEYWORDS_DIR =  ROOT_DIR + "/Data/VALIDATED_DATA/Conventions/keywords/"
TRAINING_DATA_DIR =  ROOT_DIR + "/Data/VALIDATED_DATA/Conventions/training_data/"


if not os.path.exists(BASE_DIR_SCI_ARTICLES):
    os.makedirs(BASE_DIR_SCI_ARTICLES)

if not os.path.exists(BASE_DIR_GIT_HUB):
    os.makedirs(BASE_DIR_GIT_HUB)

if not os.path.exists(BASE_DIR_AGGREGATED):
    os.makedirs(BASE_DIR_AGGREGATED)
    
if not os.path.exists(KEYWORDS_DIR):
    os.makedirs(KEYWORDS_DIR)

if not os.path.exists(TRAINING_DATA_DIR):
    os.makedirs(TRAINING_DATA_DIR)
    
    
NUM_KEYWORDS_PER_CONV = 30

In [200]:
def pre_process(text):
    
    # lowercase
    text=text.lower()
    text = text.replace("b'", "")
    
    ## Removing strings such as \\xe5 \\xe6 \\xe7 that appear a lot in the descriptions
    text = re.sub(r':?\\+x\w{2}', ' ', text, flags=re.MULTILINE)
     #text = text.replace('+',' ')
    text = re.sub('-', ' ', text, flags=re.MULTILINE) # Added by Aideen
    text = re.sub(' +', ' ', text, flags=re.MULTILINE) # Added by Aideen
    text = text.replace('+',' ')
    
    return text

# METHOD CALLED BUT NOT USED YET - MAYBE IMPLEMENT  LATER TO REMOVE SPECIAL CHARS.
#######################################################################
# using NLTK library take string, tokenize, lower case, remove stop words, lemmatize,
# stem etc and return the processed string and word frequency distribution
########################################################################
def processString(example_sent):

    ps = PorterStemmer()
    wnl = WordNetLemmatizer()

    # Combine the spanish and english stop words as some of the responses are mixed language
    # stopwords considered as noise in the text. Text may contain stop words such as
    # is, am, are, this, a, an, the, etc. In NLTK for removing stopwords, you need to create
    # a list of stopwords and filter out your list of tokens from these words.There are default
    # lists for some languages, here we use the default spanish and english stoplist(blacklist)
    spanish_stop_words = set(stopwords.words('spanish'))
    english_stop_words = set(stopwords.words('english'))
    
    stop_words = ["a"]
  
    stop_words = spanish_stop_words.union(english_stop_words)

    # Tokenization is the first step in text analytics. The process of breaking down
    # a text paragraph into smaller chunks such as words or sentence is called Tokenization.
    # Token is a single entity that is building blocks for sentence or paragraph. Here we are breaking
    # a sentence into individual words
    word_tokens = word_tokenize(example_sent)

    # convert to lower case
    word_tokens = [w.lower() for w in word_tokens]
    #print("--Tokenized and converted to lower case")

    # Only keep alpha characters, no special chars or words containing numerics
    words = [word for word in word_tokens if word.isalpha()]
    # --Words parsed to only retain alpha chars"
    numbers = [word for word in word_tokens if word.isnumeric()]
    # --Words parsed to only retain numeric chars ", numbers)
    words = numbers + words
    #print("--Words parsed: ", words) 
    
    # Remove the stop words (we are removing the english and spanish stop words)
    filtered_string = [w for w in words if not w in stop_words]
    filtered_string = []
    for w in words:
        if w not in stop_words:
            filtered_string.append(ps.stem(wnl.lemmatize(w)))
    #print("--Stopwords removed and remaining words Lemmatized and stemmed")

    fDist = FreqDist(filtered_string)
    return fDist, filtered_string

In [201]:
keyword_from_social_scientists_dict = {
    'industrial': 
        ['efficiency', 'efficiant', 'performance', 'performances', 'productivity', 'productive', 'output', 'outputs', 'control', 'power', 'functionality', 'functional', 'organisation', 'organisations', 'professional', 'professionals', 'profession', 'professions', 'reliability', 'reliable', 'foreseeability', 'qualification', 'qualifications', 'expert', 'experts', 'expertise', 'progress' , 'standard', 'standards', 'standardized', 'norms', 'norm', 'growth', 'quantification', 'quantifiy', 'measurement', 'measure', 'measures', 'engineering', 'engineer', 'forecast', 'linearity', 'extrapolation', 'extrapolate', 'scientific', 'plans', 'plan', 'audit', 'audits', 'benchmark', 'benchmarks', 'benchmarking', 'test', 'tests'],
    'project':
        ['flexibility', 'project', 'projects', 'networking', 'networker', 'networkers', 'activity', 'intermediary', 'mediation', 'switch', 'switching', 'shift', 'shifting', 'mobility', 'dynamic', 'transition', 'transitions', 'temporality', 'fluidity', 'communication', 'communications', 'communicate', 'initiative', 'initiatives', 'bridging', 'adapt', 'adaption', 'adaptions'],
    'market':
        ['wealth', 'money', 'goods', 'rival', 'selfishness', 'winner', 'winners', 'price', 'customer', 'customers', 'customized', 'seller', 'sellers', 'sell', 'vendor', 'vendors', 'buyer', 'buyers', 'buy', 'purchaser', 'purchasers', 'purchase', 'opportunity', 'opportunities', 'competition', 'compete', 'sale', 'sales', 'business', 'interest', 'transaction', 'transactions', 'availability', 'owner', 'owners', 'bargain', 'contract', 'contracts', 'pay', 'deal', 'trade', 'scarcity', 'trading', 'entrepreneur', 'entrepreneurs', 'entrepreneurial', 'market', 'markets', 'marketing', 'marketplace'],
    'inspired':
        ['inspiration', 'illuminated', 'unexpected', 'feeling', 'fascination', 'fascinating', 'thrilling', 'genuineness', 'intuition', 'genius', 'brilliant', 'uniqueness', 'non-conformity', 'non-conform', 'passion', 'passionate', 'creative', 'creativity', 'imagination', 'visionary', 'extraordinary', 'emotional', 'conviction', 'holy', 'gifted', 'blessed'],
    'civic':
        ['collective', 'collectives', 'common good', 'community', 'communities', 'representative', 'representatives', 'general interest', 'unite', 'unity', 'union', 'unions', 'majority', 'civil right', 'civil rights', 'assembly', 'democracy', 'vote', 'votes', 'voting', 'election', 'elections', 'elect', 'equality', 'fair', 'fairness', 'law', 'laws', 'justice', 'unbiased', 'impartial'],
    'domestic':
        ['superior', 'superiority', 'continuity', 'continuation', 'dependence', 'dependency', 'familiarity', 'tradition', 'traditions', 'origins', 'origin', 'routine', 'routines', 'habit', 'habits', 'etiquette', 'common sense', 'respect', 'duty', 'duties', 'trust', 'craft', 'crafts', 'craftsman', 'crafted'],
    'green':
        ['nature', 'ecological', 'ecology', 'renewable', 'recyclable', 'recycle', 'sustainable', 'sustainability', 'preserve', 'preservation', 'holism', 'holistic', 'green', 'emission', 'organic', 'organical'],
    # 'Vitality'
    'renown':
        ['acknowledgement', 'praise', 'public relation', 'public relations', 'public opinion', 'brands', 'brand', 'audience', 'follower', 'followers', 'supporter', 'supporters', 'fame', 'influence', 'influences', 'glory', 'attractive', 'appealing']
    
}

In [202]:
#obtain list of files in alphabetic order
justif_files = sorted([f for f in os.listdir(BASE_DIR_SCI_ARTICLES) if (os.path.isfile(os.path.join(BASE_DIR_SCI_ARTICLES, f)) and not f.startswith( '.' ) and not "random" in f and not "training_" in f and not "aggregated_" in f and not "splitted_" in f)])

In [203]:
#obtain list of files in alphabetic order
conv_files = sorted([f for f in os.listdir(BASE_DIR_GIT_HUB) if (os.path.isfile(os.path.join(BASE_DIR_GIT_HUB, f)) and not f.startswith( '.' ) and f.endswith( '.txt' ) and not "random" in f and not "training_" in f)])

In [204]:
print(BASE_DIR_SCI_ARTICLES)
display(justif_files)
print ("")
print(BASE_DIR_GIT_HUB)
display (conv_files)

./Data/VALIDATED_DATA/Conventions/scientific_articles/


['Civic.txt',
 'Domestic.txt',
 'Green.txt',
 'Industrial.txt',
 'Inspired.txt',
 'Market.txt',
 'Project.txt',
 'Renown.txt']


./Data/VALIDATED_DATA/Conventions/github


['Civic.txt',
 'Domestic.txt',
 'Green.txt',
 'Industrial.txt',
 'Inspired.txt',
 'Market.txt',
 'Project.txt',
 'Renown.txt']

# Agregate Scientific Article sentences with  conventions sentences

In [205]:
if FLAG__AGGREGATE_CONVENTIONS_SENTENCES_FROM_GITHUB:
    docs = []
    docsTest = []
      #iterate through each file name in BASE_DIR_SCI_ARTICLES and BASE_DIR_GITHUB 
      #create a new file aggregated_{convention name} which 
      #contains aggregated content already tokenized into sentences using sent_tokenize
    for i,x in enumerate(justif_files):
        texts = []
        source =  []
        convs = []
        
        
        print("Parsing ", x)
        with open(os.path.join(BASE_DIR_GIT_HUB, x), 'rb') as f2:
            data2 = str(f2.read()).replace("\\n", " ")
            data2 = re.sub(' +', ' ', data2)
            docs.append(pre_process(data2)) 
            #
            a, b = processString(data2)
            docsTest.append(b) 
            #
            tokenised_into_sentences = sent_tokenize(data2)
            print ("number of sentences from github: ", len(tokenised_into_sentences))
            for s in tokenised_into_sentences:
                texts.append(pre_process(s))
                source.append ("github")
                convs.append(x.replace(".txt", ""))
            f2.close() 
            
        
        with open(os.path.join(BASE_DIR_SCI_ARTICLES, x), 'rb') as f:
             #data = str(f.read())
            data = str(f.read()).replace("\\n", " ")
            data = re.sub(' +', ' ', data)
            #replace docs[i] with docs[i]+ the new data
            docs.append(docs.pop(i)+pre_process(data)) 
            a, b = processString(data)
            docsTest.append(docsTest.pop(i)+b) 
            tokenised_into_sentences = sent_tokenize(data)
            print ("number of sentences from scientific_articles ", len(tokenised_into_sentences))
            for s in tokenised_into_sentences:
                texts.append(pre_process(s))
                source.append ("sci_article")
                convs.append(x.replace(".txt", ""))
            f.close() 
        

        with open(os.path.join(
                                BASE_DIR_AGGREGATED, 
                                'aggregated_{}'.format(x.replace(".txt", ""))+".tsv"),
                                'w+') as f3: 
            f3.write("{}\t{}\t{}\n".format("sentence", "provenance", "convention"))
            for text, source, conv in zip (texts, source, convs):
                #f3.write(t.replace("\\n", "")+ '\n')
                sentence = text.replace("\\n", "").replace('\t', ' ')  # + '\n'
                f3.write("{}\t{}\t{}\n".format(sentence, source, conv.lower()))
            f3.close()
            
    #read newly created aggregated files into list sorted alphabetically      
    aggregated_files = sorted([f for f in os.listdir(BASE_DIR_AGGREGATED) if (os.path.isfile(os.path.join(BASE_DIR_AGGREGATED, f)) and not f.startswith( '.' ) and not "random" in f and not "training_" in f and "aggregated_" in f)])
    print("")
    print("New aggregated files created in folder ", BASE_DIR_AGGREGATED, ": " )
    display(aggregated_files)


Parsing  Civic.txt
number of sentences from github:  47
number of sentences from scientific_articles  174
Parsing  Domestic.txt
number of sentences from github:  20
number of sentences from scientific_articles  185
Parsing  Green.txt
number of sentences from github:  1
number of sentences from scientific_articles  85
Parsing  Industrial.txt
number of sentences from github:  309
number of sentences from scientific_articles  153
Parsing  Inspired.txt
number of sentences from github:  16
number of sentences from scientific_articles  113
Parsing  Market.txt
number of sentences from github:  100
number of sentences from scientific_articles  179
Parsing  Project.txt
number of sentences from github:  54
number of sentences from scientific_articles  300
Parsing  Renown.txt
number of sentences from github:  8
number of sentences from scientific_articles  145

New aggregated files created in folder  ./Data/VALIDATED_DATA/Conventions/aggregated/ : 


['aggregated_Civic.tsv',
 'aggregated_Domestic.tsv',
 'aggregated_Green.tsv',
 'aggregated_Industrial.tsv',
 'aggregated_Inspired.tsv',
 'aggregated_Market.tsv',
 'aggregated_Project.tsv',
 'aggregated_Renown.tsv']

In [206]:
convention_sentences_df = pd.concat([pd.read_csv(f, sep='\t') for f in glob.glob(BASE_DIR_AGGREGATED+'*.tsv')], ignore_index = True)

display(convention_sentences_df.head(5))
display(convention_sentences_df.tail(5))

,sentence,provenance,convention
0,in view of the issues with the swagger codegen...,github,civic
1,this is an easter egg concerned with the com...,github,civic
2,we would like to take this opportunity to than...,github,civic
3,"and of course, we couldn\'t do this without [o...",github,civic
4,"remember that this is a community project, pe...",github,civic


,sentence,provenance,convention
1878,retreat outside dreams leads to a fall.,sci_article,inspired
1879,beings in the state of unworthiness are define...,sci_article,inspired
1880,they are also qualified by properties that exp...,sci_article,inspired
1881,knowledge acquired through education the routi...,sci_article,inspired
1882,the polity comes undone when the temptation to...,sci_article,inspired


In [207]:
document_lengths = np.array(list(map(len, convention_sentences_df.sentence.str.split(' '))))

print("The average number of words in a document is: {}.".format(np.mean(document_lengths)))
print("The minimum number of words in a document is: {}.".format(min(document_lengths)))
print("The maximum number of words in a document is: {}.".format(max(document_lengths)))

The average number of words in a document is: 28.66171003717472.
The minimum number of words in a document is: 1.
The maximum number of words in a document is: 269.


# USE TF_IDF to create keywords per convention
TF-IDF stands for “Term Frequency — Inverse Data Frequency”. 

Term Frequency (tf): gives us the frequency of the word in each document in the corpus. It is the ratio of number of times the word appears in a document compared to the total number of words in that document. It increases as the number of occurrences of that word within the document increases. Each document has its own tf.

Inverse Data Frequency (idf): used to calculate the weight of rare words across all documents in the corpus. The words that occur rarely in the corpus have a high IDF score. 

For equations see:
https://www.freecodecamp.org/news/how-to-process-textual-data-using-tf-idf-in-python-cd2bbc0a94a3/

In [208]:
##### 
#To test this we need to first reconvert the words into one string. 
#docs = docsTest
########
cv=CountVectorizer(max_df=0.85,max_features=10000)
word_count_vector=cv.fit_transform(docs)
list(cv.vocabulary_.keys())[:10]

['view',
 'issues',
 'swagger',
 'codegen',
 'beta',
 'release',
 'disagreement',
 'direction',
 '40',
 'top']

In [209]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [210]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results
 

In [211]:
# you only needs to do this once, this is a mapping of index to 
feature_names=cv.get_feature_names()

keywords_for_df = []
convention_for_df = []
for i,f in enumerate(aggregated_files):
    # get the document that we want to extract keywords from
    doc = docs[i]
    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 30
    keywords=extract_topn_from_vector(feature_names,sorted_items,NUM_KEYWORDS_PER_CONV)
    

    ss_keys = keyword_from_social_scientists_dict[f.replace('aggregated_', '').replace('.tsv','').lower()]
    # now add the result to dataframe
    for k in keywords:
        convention_for_df.append(f.replace('aggregated_', '').replace('.tsv','').lower())
        keywords_for_df.append(k)
    for key in ss_keys:
        convention_for_df.append(f.replace('aggregated_', '').replace('.tsv','').lower())
        keywords_for_df.append(key)

zippedList =  list(zip(keywords_for_df, convention_for_df))
keywords_df = pd.DataFrame(zippedList, columns = ['keyword' , 'convention'])
keywords_df = keywords_df.drop_duplicates()
display (keywords_df.head(5))

,keyword,convention
0,community,civic
1,collective,civic
2,civic,civic
3,solidarity,civic
4,interests,civic


# Store keywords as CSV file 

In [212]:
path = KEYWORDS_DIR+'keywords.csv'
display ("storing keywords.csv at path:", path)
keywords_df.to_csv(path,index = None, header=True)

'storing keywords.csv at path:'

'./Data/VALIDATED_DATA/Conventions/keywords/keywords.csv'

# Create training data DF and Save as CSV

In [213]:
#Apply one hot encoding to column "convention"
print("Applying one hot encoding to convention")
convention_sentences_df["convention"] = pd.Categorical(convention_sentences_df["convention"])
dfDummies = pd.get_dummies(convention_sentences_df["convention"], prefix="convention")
convention_sentences_df = pd.concat([convention_sentences_df, dfDummies], axis=1)
convention_sentences_df = convention_sentences_df.drop(["convention"], axis=1)
display (convention_sentences_df.head(5))
convention_sentences_df.to_csv(TRAINING_DATA_DIR+'training_data.csv')






Applying one hot encoding to convention


,sentence,provenance,convention_civic,convention_domestic,convention_green,convention_industrial,convention_inspired,convention_market,convention_project,convention_renown
0,in view of the issues with the swagger codegen...,github,1,0,0,0,0,0,0,0
1,this is an easter egg concerned with the com...,github,1,0,0,0,0,0,0,0
2,we would like to take this opportunity to than...,github,1,0,0,0,0,0,0,0
3,"and of course, we couldn\'t do this without [o...",github,1,0,0,0,0,0,0,0
4,"remember that this is a community project, pe...",github,1,0,0,0,0,0,0,0


# STOP HERE (REVIEW REST OF CODE TO CREATE A panda DF per model and dump each to a csv file rather than several read/write to files )

## Create the training data per model

In [ ]:
## This code generates a training file for each characteristic by 
## concetanting sentences for convention X annotated with label one 
## and sentences from all the other convention that are annotated with label 0.


#Generate the data frames for each model
#df_by_convention = df[df.convention == 'Green']
#display (df_by_convention.sample(n = 3))


##It generates a training_{convention_name}.csv file for each convention
def generate_training_files(data_dir, data_files):
    for file_name in data_files:

        f = open(os.path.join(data_dir,file_name), "r")
        original_lines = f.readlines()
        
        original_lines2=[]
        for l in original_lines:
                for l2 in l.split("."):
                    original_lines2.append(l2)
        f.close()


        """
        Step 1: Generate four random numbers between 0 and 1
        Step 2: Add these four numbers
        Step 3: Divide each of the four numbers by the sum,
        Step 4: Multiply by 100, and round to the nearest integer.
        """
        num_original_lines = len(original_lines2)*2

        num_random_lines = np.random.uniform(0,1,len(data_files))
        num_random_lines = (num_random_lines / sum(num_random_lines))*num_original_lines

        tmp_all_files = [f for f in data_files if f != file_name]

        random_lines = []    
        for idx, file_name2 in enumerate(tmp_all_files):
            f2 = open(os.path.join(data_dir,file_name2), "r")
            tmp_lines = f2.readlines()
            
            tmp_lines2 = []
            for l in tmp_lines:
                for l2 in l.split("."):
                    tmp_lines2.append(l2)
            f2.close()

            for i in range(int(num_random_lines[idx])):
                random_lines.append(random.choice(tmp_lines2))

        with open(rreplace(os.path.join(data_dir,file_name), "/", "/training_",1), "w") as f3:
            f3.write("{}\t{}\n".format("category", "text"))

            for idx, l in enumerate(original_lines2):
                l = l.replace('\n', ' ').replace('\t', ' ')
                f3.write("{}\t\"{}\"\n".format(1, l))


            for idx, l2 in enumerate(random_lines):
                l2 = l2.replace('\n', '').replace('\t', ' ')
                f3.write("{}\t\"{}\"\n".format(0, l2))
            f3.close()